In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
pd.pandas.set_option('display.max_columns',None)


df_train = pd.read_csv("train.txt", delimiter=';', header=None, names=['Text', 'Emotion'])



In [2]:
print(df_train.shape)
df_train.head()

(16000, 2)


,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [3]:
df_test = pd.read_csv("test.txt", delimiter = ';', header=None, names = ['Text', 'Emotion'])

In [4]:

print(df_test.shape)
df_test.head()

(2000, 2)


,Text,Emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [5]:
df_val = pd.read_csv("val.txt", delimiter = ';', header=None, names = ['Text', 'Emotion'])
print(df_val.shape)
df_val.head()

(2000, 2)


,Text,Emotion
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [6]:
Result_df = pd.concat([df_train,df_test])

In [7]:
print(Result_df.shape)
Result_df.head()

(18000, 2)


,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [8]:
Result = pd.concat([Result_df,df_val])
print(Result.shape)
Result.head()

(20000, 2)


,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [9]:
Result['Emotion'].value_counts()

joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: Emotion, dtype: int64

In [10]:
Result['Emotion_value'] = Result['Emotion'].map({
    'joy':0,
    'sadness':1,
    'anger':2,
    'fear':3,
    'love':4,
    'surprise':5
})

Result.head()
    

,Text,Emotion,Emotion_value
0,i didnt feel humiliated,sadness,1
1,i can go from feeling so hopeless to so damned...,sadness,1
2,im grabbing a minute to post i feel greedy wrong,anger,2
3,i am ever feeling nostalgic about the fireplac...,love,4
4,i am feeling grouchy,anger,2


In [11]:
Result['Emotion_value'].value_counts()

0    6761
1    5797
2    2709
3    2373
4    1641
5     719
Name: Emotion_value, dtype: int64

In [12]:
import spacy

# load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm") 

def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 
    

Result['Preprocess_Text'] = Result['Text'].apply(preprocess)

In [13]:
Result['Emotion'].value_counts()

joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: Emotion, dtype: int64

In [14]:
Result['Emotion_value'].dtypes

dtype('int64')

In [15]:
Result.head()

,Text,Emotion,Emotion_value,Preprocess_Text
0,i didnt feel humiliated,sadness,1,not feel humiliate
1,i can go from feeling so hopeless to so damned...,sadness,1,feel hopeless damned hopeful care awake
2,im grabbing a minute to post i feel greedy wrong,anger,2,m grab minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,love,4,feel nostalgic fireplace know property
4,i am feeling grouchy,anger,2,feel grouchy


In [28]:
Result_ = Result[['Preprocess_Text', 'Emotion_value']].copy()


In [29]:
Result_.head()

,Preprocess_Text,Emotion_value
0,not feel humiliate,1
1,feel hopeless damned hopeful care awake,1
2,m grab minute post feel greedy wrong,2
3,feel nostalgic fireplace know property,4
4,feel grouchy,2


In [31]:
Result_['Emotion_value'].value_counts()

0    6761
1    5797
2    2709
3    2373
4    1641
5     719
Name: Emotion_value, dtype: int64

In [32]:
Result_.dtypes

Preprocess_Text    object
Emotion_value       int64
dtype: object

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
import pandas as pd

# Assuming 'data' is your DataFrame with 'Text' and 'Emotion' columns
X = Result_['Preprocess_Text']
y = Result_['Emotion_value']

# Create a ColumnTransformer to apply CountVectorizer to the 'Text' column
preprocessor = ColumnTransformer(
    transformers=[
        ('text', CountVectorizer(), 'Preprocess_Text')
    ],
    remainder='passthrough'
)

# Fit and transform the data using the ColumnTransformer
X_transformed = preprocessor.fit_transform(Result_)

# Convert the transformed data into a DataFrame (optional)
transformed_df = pd.DataFrame(X_transformed)

# Print the transformed DataFrame
print(transformed_df)


                                                       0
0        (0, 4245)\t1\n  (0, 5591)\t1\n  (0, 8028)\t1...
1        (0, 812)\t1\n  (0, 1672)\t1\n  (0, 2754)\t1\...
2        (0, 4245)\t1\n  (0, 4957)\t1\n  (0, 5011)\t1...
3        (0, 4245)\t1\n  (0, 4355)\t1\n  (0, 6521)\t1...
4          (0, 4245)\t1\n  (0, 5046)\t1\n  (0, 13289)\t2
...                                                  ...
19995    (0, 2165)\t1\n  (0, 4000)\t3\n  (0, 4001)\t1...
19996    (0, 1287)\t1\n  (0, 2371)\t1\n  (0, 3051)\t1...
19997    (0, 4072)\t1\n  (0, 4245)\t1\n  (0, 5754)\t1...
19998    (0, 4245)\t1\n  (0, 8541)\t1\n  (0, 11118)\t...
19999    (0, 1563)\t1\n  (0, 2711)\t1\n  (0, 4245)\t1...

[20000 rows x 1 columns]


In [ ]:
import pandas as pd

# Assuming 'data' is your DataFrame with 'Text' and 'Emotion' columns

# Filter the DataFrame to select only 50 samples for each emotion
happy_samples = Result[Result['Emotion'] == 'happy'].head(50)
sad_samples = data[data['Emotion'] == 'sad'].head(50)
surprise_samples = data[data['Emotion'] == 'surprise'].head(50)
angry_samples = data[data['Emotion'] == 'angry'].head(50)

# Concatenate the filtered subsets together to create a new DataFrame
new_data = pd.concat([happy_samples, sad_samples, surprise_samples, angry_samples], ignore_index=True)

# Select only the 'Text' column
new_data = new_data[['Text', 'Emotion']]

# Print the new dataset
print(new_data)


In [45]:
happy_samples = Result[Result['Emotion'] == 'joy'].head(50)
happy_samples.head()

,Text,Emotion,Emotion_value,Preprocess_Text
8,i have been with petronas for years i feel tha...,joy,0,petrona year feel petrona perform huge profit
11,i do feel that running is a divine experience ...,joy,0,feel running divine experience expect type spi...
14,i have immense sympathy with the general point...,joy,0,immense sympathy general point possible proto ...
15,i do not feel reassured anxiety is on each side,joy,0,feel reassure anxiety
22,i have the feeling she was amused and delighted,joy,0,feeling amuse delight


In [40]:
print(happy_samples)

Empty DataFrame
Columns: [Text, Emotion, Emotion_value, Preprocess_Text]
Index: []


In [44]:
Result.head()

,Text,Emotion,Emotion_value,Preprocess_Text
0,i didnt feel humiliated,sadness,1,not feel humiliate
1,i can go from feeling so hopeless to so damned...,sadness,1,feel hopeless damned hopeful care awake
2,im grabbing a minute to post i feel greedy wrong,anger,2,m grab minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,love,4,feel nostalgic fireplace know property
4,i am feeling grouchy,anger,2,feel grouchy


In [46]:
Result['Emotion'].value_counts()

joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: Emotion, dtype: int64

In [47]:
import pandas as pd

# Assuming 'data' is your DataFrame with 'Text' and 'Emotion' columns

# Filter the DataFrame to select only 50 samples for each emotion
joy_samples = Result[Result['Emotion'] == 'joy'].head(700)
sad_samples = Result[Result['Emotion'] == 'sadness'].head(700)
anger_samples = Result[Result['Emotion'] == 'anger'].head(700)
fear_samples = Result[Result['Emotion'] == 'fear'].head(700)
love_samples = Result[Result['Emotion'] == 'love'].head(700)
surprise_samples = Result[Result['Emotion'] == 'surprise'].head(700)



# Concatenate the filtered subsets together to create a new dataset
new_data = pd.concat([joy_samples, sad_samples, anger_samples, fear_samples, love_samples, surprise_samples], ignore_index=True)

# Print the new dataset
print(new_data)


                                                   Text   Emotion  \
0     i have been with petronas for years i feel tha...       joy   
1     i do feel that running is a divine experience ...       joy   
2     i have immense sympathy with the general point...       joy   
3       i do not feel reassured anxiety is on each side       joy   
4       i have the feeling she was amused and delighted       joy   
...                                                 ...       ...   
4195  im watching my sodium which mostly means im fe...  surprise   
4196  i was willing to be honest with myself and put...  surprise   
4197  i could get fucking stuffed in fucking chocola...  surprise   
4198  i am looking forward to how amazing it makes m...  surprise   
4199  im not going to lie it feels really weird to b...  surprise   

      Emotion_value                                    Preprocess_Text  
0                 0      petrona year feel petrona perform huge profit  
1                 0  feel

In [48]:
new_data.head()

,Text,Emotion,Emotion_value,Preprocess_Text
0,i have been with petronas for years i feel tha...,joy,0,petrona year feel petrona perform huge profit
1,i do feel that running is a divine experience ...,joy,0,feel running divine experience expect type spi...
2,i have immense sympathy with the general point...,joy,0,immense sympathy general point possible proto ...
3,i do not feel reassured anxiety is on each side,joy,0,feel reassure anxiety
4,i have the feeling she was amused and delighted,joy,0,feeling amuse delight


In [50]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Text             4200 non-null   object
 1   Emotion          4200 non-null   object
 2   Emotion_value    4200 non-null   int64 
 3   Preprocess_Text  4200 non-null   object
dtypes: int64(1), object(3)
memory usage: 131.4+ KB


In [51]:
# so the data is balanced so we can give emphasis to accuracy in classification reports

# we will preoceed with train test split algorithm

from sklearn.model_selection import train_test_split
X = new_data['Preprocess_Text']
y = new_data['Emotion_value']

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.20, random_state=101, stratify=new_data.Emotion_value)
 
# Importing the libraries just for differentiating between different algorithms

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Random Forest 


Random_clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('RF', RandomForestClassifier(n_estimators=50, criterion='entropy', random_state=101))
])    
    
Random_clf.fit(X_train, y_train)

y_pred = Random_clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.55      0.62       140
           1       0.62      0.75      0.68       140
           2       0.82      0.80      0.81       140
           3       0.89      0.73      0.80       140
           4       0.80      0.91      0.85       140
           5       0.89      0.96      0.92       140

    accuracy                           0.78       840
   macro avg       0.79      0.78      0.78       840
weighted avg       0.79      0.78      0.78       840



In [52]:
# K nearest Neighbors Classifier

from  sklearn.neighbors import KNeighborsClassifier

KNN_clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('KNN', KNeighborsClassifier(n_neighbors=5,metric='euclidean'))
])    

KNN_clf.fit(X_train, y_train)
y_pred = KNN_clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.35      0.35      0.35       140
           1       0.36      0.49      0.41       140
           2       0.45      0.47      0.46       140
           3       0.50      0.42      0.46       140
           4       0.60      0.38      0.46       140
           5       0.68      0.75      0.71       140

    accuracy                           0.48       840
   macro avg       0.49      0.48      0.48       840
weighted avg       0.49      0.48      0.48       840



In [53]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB

NB_clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('NB', MultinomialNB())
])    

NB_clf.fit(X_train, y_train)
y_pred = NB_clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.66      0.70       140
           1       0.71      0.61      0.66       140
           2       0.72      0.74      0.73       140
           3       0.70      0.67      0.68       140
           4       0.76      0.87      0.81       140
           5       0.80      0.91      0.85       140

    accuracy                           0.74       840
   macro avg       0.74      0.74      0.74       840
weighted avg       0.74      0.74      0.74       840



## Multinomial Naive Bayes has performed best.